In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY=os.getenv("TAVILY_API_KEY")
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY=os.getenv("LANGCHAIN_API_KEY")
LANGCHAIN_PROJECT=os.getenv("LANGCHAIN_PROJECT")

In [3]:
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"]= GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"]=LANGCHAIN_PROJECT

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [8]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-1.0-pro")

In [6]:
'''from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")'''
from langchain_groq import ChatGroq
import os
llm=ChatGroq(model_name="Gemma2-9b-It")

# Simple AI Assistant

In [9]:
while True:
    question=input("type your question. if you want to quit the chat write quit")
    if question !="quit":
        print(llm.invoke(question).content)
    else:
        print("goodbye take care yourself")
        break

Hello there! How can I assist you today?
I do not have real-time access to the internet and my knowledge cutoff is April 2023. Therefore, I cannot provide you with the current weather conditions. 

To get the most up-to-date and accurate weather forecast, I recommend checking a reputable weather app or website, such as the Weather Channel or AccuWeather.
goodbye take care yourself


In [10]:
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.messages import AIMessage

In [11]:
store={}

In [12]:
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

In [13]:
config = {"configurable": {"session_id": "firstchat"}}

In [14]:
model_with_memory=RunnableWithMessageHistory(llm,get_session_history)

In [15]:
model_with_memory.invoke(("Hi! I'm Santosh Kumar"),config=config).content

"Hello Santosh Kumar, it's nice to meet you!"

In [16]:

model_with_memory.invoke(("tell me what is my name?"),config=config).content

'Your name is Santosh Kumar. You told me this earlier in the conversation.'

In [17]:
store

{'firstchat': InMemoryChatMessageHistory(messages=[HumanMessage(content="Hi! I'm Santosh Kumar", additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Santosh Kumar, it's nice to meet you!", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-66636ae2-e6e4-459b-aa2f-db9e78309dde-0', usage_metadata={'input_tokens': 9, 'output_tokens': 13, 'total_tokens': 22}), HumanMessage(content='tell me what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Santosh Kumar. You t

# RAG with LCEL

In [19]:
from langchain_community.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough , RunnableLambda
from langchain_core.output_parsers import StrOutputParser

### Reading the txt files from source directory

loader = DirectoryLoader('../data', glob="./*.txt", loader_cls=TextLoader)
docs = loader.load()

### Creating Chunks using RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=50,
    chunk_overlap=10,
    length_function=len
)
new_docs = text_splitter.split_documents(documents=docs)
doc_strings = [doc.page_content for doc in new_docs]

###  BGE Embddings

'''from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)
'''

### Creating Retriever using Vector DB

db = Chroma.from_documents(new_docs, embeddings)
retriever = db.as_retriever(search_kwargs={"k": 4})

In [20]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [21]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
    )

In [22]:
question ="what is llama3? can you highlight 3 important points?"
print(retrieval_chain.invoke(question))

1. Llama 3 is a large language model developed by Google.
2. Llama 3 was released in April 2024.
3. Llama 3 is used by various services, including Gemini and Google Search.


# Let's Start with Tools and Agents

In [23]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [24]:
api_wrapper = WikipediaAPIWrapper(top_k_results=5, doc_content_chars_max=100)

In [25]:
tool = WikipediaQueryRun(api_wrapper=api_wrapper)

In [26]:
tool.name

'wikipedia'

In [27]:
tool.description

'A wrapper around Wikipedia. Useful for when you need to answer general questions about people, places, companies, facts, historical events, or other subjects. Input should be a search query.'

In [28]:
tool.args

{'query': {'description': 'query to look up on wikipedia',
  'title': 'Query',
  'type': 'string'}}

In [29]:
tool.return_direct

False

In [30]:
print(tool.run({"query": "langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of 


In [31]:
tool.run("langchain")

'Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of '

In [38]:
from langchain_community.tools import YouTubeSearchTool

In [39]:
tool2=YouTubeSearchTool()

In [40]:
tool2.name

'youtube_search'

In [44]:
tool2.run("freecodecamp")

"['https://www.youtube.com/watch?v=iSiyDHobXHA&pp=ygUMZnJlZWNvZGVjYW1w', 'https://www.youtube.com/watch?v=BV03sQ0srcU&pp=ygUMZnJlZWNvZGVjYW1w']"

In [33]:
from langchain_core.pydantic_v1 import BaseModel, Field

In [34]:
class WikiInputs(BaseModel):
    query: str = Field(description="query to look up in Wikipedia, should be 3 or less words")

In [46]:
from langchain_community.tools.tavily_search import TavilySearchResults

In [47]:
tool = TavilySearchResults()

In [49]:
tool.invoke({"query": "What happened in the latest burning man floods"})

[{'url': 'https://www.npr.org/2023/09/03/1197497458/the-latest-on-the-burning-man-flooding',
  'content': "There are also reports that at least one person has died at the counterculture festival about a hundred miles north of Reno, Nev. Earlier this afternoon, I caught up with NPR's Claudia Peschiutta, who's at her first burn, and she told me it's muddy where she is, but that she and her camp family have been making the best of things.\n National\nThe latest on the Burning Man flooding\nClaudia Peschiutta\nAuthorities are investigating a death at the Burning Man festival in the Nevada desert after tens of thousands of people are stuck in camps because of rain.\n SCOTT DETROW, HOST:\nKnee-deep mud, warnings to conserve food and water, orders to shelter in place - this is all at Burning Man 2023 after torrential rains turned the Black Rock Desert into miles and miles of mud. I mean, mostly what I've seen from my personal experience is just any sort of need that you have, somebody, whethe

In [50]:
from langchain.agents import AgentType
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [51]:
tool=load_tools(["wikipedia"],llm=llm)

In [53]:
agent = initialize_agent(tool,llm,agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)

In [54]:
agent.run("What is the current GDP of India?")

C:\Users\santo\AppData\Local\Temp\ipykernel_7644\4101983600.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("What is the current GDP of India?")




> Entering new AgentExecutor chain...
Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Thought:Action: [wikipedia]
Action Input: What is the GDP of India?
Observation: [wikipedia] is not a valid tool, try one of [wikipedia].
Th

'Agent stopped due to iteration limit or time limit.'